In [ ]:
import intake
from easygems import healpix as egh

import numpy as np

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Time period

time = ('2020-08-01','2020-08-31')

# Region

domains10x10 = {
    "peruvian":     np.array([-90, -80, -20, -10]) ,
    "namibian":     np.array([0, 10, -20, -10]),
    "californian":  np.array([-130, -120, 20, 30]),
    "canarian":     np.array([-35, -25, 15, 25])
}

map_domain = domains10x10['namibian'] + np.array([-1,1,-1,1])*0

In [ ]:
# Load dataset

hknode = 'EU'
sim = 'icon_d3hp003'
zoom = 11


cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")[hknode]

ds_globe = cat[sim](time_method='inst',time='PT6H',zoom=zoom).to_dask().pipe(egh.attach_coords)

In [ ]:
# Select time period and and region

cells = egh.isel_extent(ds_globe,map_domain+np.array([-1,1,-1,1])*0) # margin on each side

ds = ds_globe.sel(time=slice(*time)).isel(cell=cells)

In [ ]:
# Check average height of pressure levels

zgm = ds['zg'].mean(dim=('time','cell'))

plt.plot(ds['pressure']/100,zgm,'o')
plt.grid()
plt.ylim(0,2.1e3)
plt.xlim(800,1000)
plt.xlabel('Pressure [hPa]')
plt.ylabel('Height [m]')

In [ ]:
%%time

# Derive relevant parameters


# Integrate qall
ds['qallvi'] = (ds['qall'].integrate('pressure')/9.81) \
    .assign_attrs(long_name='Total condensate', units='kg/m^2')


# Find max and min column velocity
ds['wamax'] = ds['wa'].sel(pressure=slice(900e2,1000e2)).max(dim='pressure') \
        .assign_attrs(long_name='Max vert velocity below 900 hPa', units='m/2')

ds['wamin'] = ds['wa'].sel(pressure=slice(900*100,1000*100)).min(dim='pressure') \
        .assign_attrs(long_name='Min vert velocity below 900 hPa', units='m/2')


# Change units for precip
ds['pr'].data = ds['pr'].data*3600*24/1e3*1e3
ds['pr'].attrs['units'] = 'mm/day'

In [ ]:
# Map plotting functions

def draw_map(ax,map_domain):
    ax.set_extent(map_domain, crs=ccrs.PlateCarree())
    ax.add_feature(cf.NaturalEarthFeature('physical', 'land', '10m'),
               facecolor='none', edgecolor='red', linewidth=1)
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

def plot_annotate(ax,da):
    cb = plt.colorbar(im, ax=ax, shrink=0.9, aspect=30, pad=0.02,
                      label=f"{da.attrs['long_name']:s} [{da.attrs['units']:s}]")
    datestr = da.time.values.astype('datetime64[h]').item().strftime('%Y-%m-%dT%H')
    if 'healpix_nside' in da.crs.attrs:
        zoom = np.log2(da.crs.healpix_nside).astype(int)
        ax.set_title(datestr+f"  zoom={zoom:d}")
    else:
        ax.set_title(datestr)

In [ ]:
# Plot example maps

time_step = np.timedelta64(10,'D')

plot_path = f"./figures/map_ex/"
# plot_path = f"/work/bb1153/b382422/global_hackathon/zoom{zoom:d}_4x4/"
figsize = (14,4)

for t in np.arange(ds.time[0].values,ds.time[-1].values,time_step):

    fig, axs = plt.subplots(1,3,figsize=figsize,
                            subplot_kw={"projection": ccrs.PlateCarree()},
                            constrained_layout=True)

    da = ds['qallvi'].sel(time=t)
    ax = axs[0]
    draw_map(ax,map_domain)
    im = egh.healpix_show(da, ax=ax, cmap='Blues_r', vmin=0, vmax=1)
    plot_annotate(ax,da)

    da = ds['wamax'].sel(time=t)
    ax = axs[1]
    draw_map(ax,map_domain)
    im = egh.healpix_show(da, ax=ax, cmap='bwr', vmin=-0.4, vmax=0.4)
    plot_annotate(ax,da)

    da = ds['pr'].sel(time=t)
    ax = axs[2]
    draw_map(ax,map_domain)
    im = egh.healpix_show(da, ax=ax, cmap='Purples',vmin=0, vmax=25)
    plot_annotate(ax,da)

    datestr = t.astype('datetime64[h]').item().strftime('%Y-%m-%dT%H')
    plt.savefig(plot_path+datestr,bbox_inches='tight',dpi=600)

    print(datestr)
    # plt.close(fig)